https://adventofcode.com/2020/day/23

In [1]:
input = r'''247819356'''

In [2]:
test_input = r'''389125467'''

In [3]:
from copy import copy
from tqdm import tqdm

In [4]:
def iteration(nums):
  nums_max = max(nums)
  nums = copy(nums)
  held = nums[1:4]
  nums = [nums[0]] + nums[4:]
  current_cup = nums[0]
  looking_for = current_cup -1
  while looking_for not in nums:
    looking_for -= 1
    if looking_for == 0 or looking_for == -1:
      looking_for = nums_max
  destination_index = nums.index(looking_for)
  nums = nums[:destination_index+1] + held + nums[destination_index+1:]
  nums = nums[1:] + [nums[0]]
  return nums

def main(input, times=100, debug=False):
  nums = [int(d) for d in input]
  
  for _ in range(times):
    if debug:
      print(nums)
    nums = iteration(nums)

  idx = nums.index(1)
  ret = ''.join([str(n) for n in nums])
  return ret[idx+1:] + ret[:idx]

In [5]:
main(test_input, 10, debug=True)

[3, 8, 9, 1, 2, 5, 4, 6, 7]
[2, 8, 9, 1, 5, 4, 6, 7, 3]
[5, 4, 6, 7, 8, 9, 1, 3, 2]
[8, 9, 1, 3, 4, 6, 7, 2, 5]
[4, 6, 7, 9, 1, 3, 2, 5, 8]
[1, 3, 6, 7, 9, 2, 5, 8, 4]
[9, 3, 6, 7, 2, 5, 8, 4, 1]
[2, 5, 8, 3, 6, 7, 4, 1, 9]
[6, 7, 4, 1, 5, 8, 3, 9, 2]
[5, 7, 4, 1, 8, 3, 9, 2, 6]


'92658374'

In [6]:
main(input, 100)

'76385429'

https://adventofcode.com/2020/day/23#part2

Initially, we had the idea to represent the list of numbers with tuples representing ranges of consecutive numbers. Alina realized that the consecutive ranges would be broken up quickly, so this would not be useful. Also, the code for this was surprisingly laborious to write.

We knew that naively running our code from part one would be much too slow, but we wrote the following cell to try it out:

In [7]:
def iteration(nums):
  nums_max = max(nums)
  nums = copy(nums)
  held = nums[1:4]
  nums = [nums[0]] + nums[4:]
  current_cup = nums[0]
  looking_for = current_cup -1
  while looking_for not in nums:
    looking_for -= 1
    if looking_for == 0 or looking_for == -1:
      looking_for = nums_max
  destination_index = nums.index(looking_for)
  nums = nums[:destination_index+1] + held + nums[destination_index+1:]
  nums = nums[1:] + [nums[0]]
  return nums

def main(input, times=100, debug=False):
  nums = [int(d) for d in input]
  nums += list(range(max(nums)+1, 1000000 + 1))
  if debug:
    print(max(nums))
  
  for _ in tqdm(range(times), position=0, leave=True):
    # if debug:
    #   print(nums)
    nums = iteration(nums)

  one_index = nums.index(1)

  return nums[(one_index+1)%len(nums)] * nums[(one_index+2)%len(nums)]

In [8]:
main(input, times=10000000)

  0%|          | 1746/10000000 [03:39<348:48:22,  7.96it/s]

KeyboardInterrupt: ignored

Finally we realized we should used a circularly linked list! We use a doubly linked list here, but we suspect we could have used a singularly linked list.

In [9]:
def update(linked, current_cup, debug=False):
  nums_max = len(linked) - 1

  # build the set of "held" or set aside cups
  f = linked[current_cup][1]
  s = linked[f][1]
  t = linked[s][1]
  after = linked[t][1]
  linked[current_cup][1] = after
  linked[after][0] = current_cup

  # find the location where the held set will be inserted
  looking_for = current_cup - 1
  if looking_for < 1:
    looking_for = nums_max
  while looking_for in (f, s, t):
    looking_for -= 1
    if looking_for < 1:
      looking_for = nums_max

  # insert the held set back in
  after_insert = linked[looking_for][1]
  linked[looking_for][1] = f
  linked[f][0] = looking_for
  linked[t][1] = after_insert
  linked[after_insert][0] = t

  # update the current cup
  current_cup = linked[current_cup][1]

  # return the results
  return linked, current_cup

def main(input, times=100, debug=False):
  nums = [int(d) for d in input]
  nums += list(range(max(nums)+1, 1000000 + 1))

  linked = [None] * (len(nums) + 1)
  for i, n in enumerate(nums):
    linked[n] = [nums[(i-1)%len(nums)], nums[(i+1)%len(nums)]]

  current_cup = nums[0]
  
  for _ in tqdm(range(times), position=0, leave=True):
    linked, current_cup = update(linked, current_cup, debug=debug)

  after_one = linked[1][1]
  after_after_one = linked[after_one][1]

  return after_one * after_after_one

In [10]:
main(test_input, times=10000000)

100%|██████████| 10000000/10000000 [00:22<00:00, 441474.51it/s]


149245887792

In [11]:
main(input, times=10000000)

100%|██████████| 10000000/10000000 [00:23<00:00, 427024.92it/s]


12621748849